In [15]:
import os
import io
import json
import torch
import numpy as np
from collections import defaultdict
from torch.utils.data import Dataset
from nltk.tokenize import TweetTokenizer
import collections

In [9]:
data_file = '../data/clean_paraphrases.tsv'
q1_data = []

with open(data_file, 'r') as f:
    next(f)
    for line in f:
        line = line.strip().split('\t')
        q1_data.append(line[0])

In [10]:
len(q1_data)

149263

In [11]:
q1_data[:10]

['astrology : i am a capricorn sun cap moon and cap rising ... what does that say about me ?',
 'how can i be a good geologist ?',
 'how do i read and find my youtube comments ?',
 'what can make physics easy to learn ?',
 'what was your first sexual experience like ?',
 'what would a trump presidency mean for current international master ’ s students on an f1 visa ?',
 'what does manipulation mean ?',
 'why are so many quora users posting questions that are readily answered on google ?',
 'why do rockets look white ?',
 'how should i prepare for ca final law ?']

In [14]:
with open('data/qqp.train.txt', 'w') as f:
    for line in q1_data[:-4000]:
        f.write(line + '\n') 
        
with open('data/qqp.valid.txt', 'w') as f:
    for line in q1_data[-4000:]:
        f.write(line + '\n') 

In [27]:
class QQP(Dataset):

    def __init__(self, data_dir, split, create_data, **kwargs):

        super().__init__()
        self.data_dir = data_dir
        self.split = split
        self.max_sequence_length = kwargs.get('max_sequence_length', 20)
        self.min_occ = kwargs.get('min_occ', 3)

        self.raw_data_path = os.path.join(data_dir, 'qqp.'+ split +'.txt')
        self.data_file = 'qqp.'+ split +'.json'
        self.vocab_file = 'qqp.vocab.json'

        if create_data:
            print("Creating new %s qqp data."%split.upper())
            self._create_data()

        elif not os.path.exists(os.path.join(self.data_dir, self.data_file)):
            print("%s preprocessed file not found at %s. Creating new."%(split.upper(), os.path.join(self.data_dir, self.data_file)))
            self._create_data()

        else:
            self._load_data()


    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        idx = str(idx)

        return {
            'input': np.asarray(self.data[idx]['input']),
            'target': np.asarray(self.data[idx]['target']),
            'length': self.data[idx]['length']
        }

    @property
    def vocab_size(self):
        return len(self.w2i)

    @property
    def pad_idx(self):
        return self.w2i['<pad>']

    @property
    def sos_idx(self):
        return self.w2i['<sos>']

    @property
    def eos_idx(self):
        return self.w2i['<eos>']

    @property
    def unk_idx(self):
        return self.w2i['<unk>']

    def get_w2i(self):
        return self.w2i

    def get_i2w(self):
        return self.i2w


    def _load_data(self, vocab=True):

        with open(os.path.join(self.data_dir, self.data_file), 'r') as file:
            self.data = json.load(file)
        if vocab:
            with open(os.path.join(self.data_dir, self.vocab_file), 'r') as file:
                vocab = json.load(file)
            self.w2i, self.i2w = vocab['w2i'], vocab['i2w']

    def _load_vocab(self):
        with open(os.path.join(self.data_dir, self.vocab_file), 'r') as vocab_file:
            vocab = json.load(vocab_file)

        self.w2i, self.i2w = vocab['w2i'], vocab['i2w']

    def _create_data(self):

        if self.split == 'train':
            self._create_vocab()
        else:
            self._load_vocab()

        tokenizer = TweetTokenizer(preserve_case=False)

        data = defaultdict(dict)
        with open(self.raw_data_path, 'r') as f:

            for i, line in enumerate(f):

                words = tokenizer.tokenize(line)

                input = ['<sos>'] + words
                input = input[:self.max_sequence_length]

                target = words[:self.max_sequence_length-1]
                target = target + ['<eos>']

                assert len(input) == len(target), "%d, %d"%(len(input), len(target))
                length = len(input)

                input.extend(['<pad>'] * (self.max_sequence_length-length))
                target.extend(['<pad>'] * (self.max_sequence_length-length))

                input = [self.w2i.get(w, self.w2i['<unk>']) for w in input]
                target = [self.w2i.get(w, self.w2i['<unk>']) for w in target]

                id = len(data)
                data[id]['input'] = input
                data[id]['target'] = target
                data[id]['length'] = length

        with io.open(os.path.join(self.data_dir, self.data_file), 'wb') as data_file:
            data = json.dumps(data, ensure_ascii=False)
            data_file.write(data.encode('utf8', 'replace'))

        self._load_data(vocab=False)

    def _create_vocab(self):

        assert self.split == 'train', "Vocablurary can only be created for training file."

        tokenizer = TweetTokenizer(preserve_case=False)

        wordcounts = collections.Counter()
        w2i = dict()
        i2w = dict()

        special_tokens = ['<pad>', '<unk>', '<sos>', '<eos>']
        for st in special_tokens:
            i2w[len(w2i)] = st
            w2i[st] = len(w2i)

        with open(self.raw_data_path, 'r') as f:

            for line in f:
                for word in tokenizer.tokenize(line):
                    wordcounts[word] += 1
                
            for w, c in dict(wordcounts.most_common()).items():
                if c > self.min_occ and w not in special_tokens:
                    i2w[len(w2i)] = w
                    w2i[w] = len(w2i)

        assert len(w2i) == len(i2w)

        print("Vocablurary of %i keys created." %len(w2i))

        vocab = dict(w2i=w2i, i2w=i2w)
        with io.open(os.path.join(self.data_dir, self.vocab_file), 'wb') as vocab_file:
            data = json.dumps(vocab, ensure_ascii=False)
            vocab_file.write(data.encode('utf8', 'replace'))

        self._load_vocab()

In [29]:
train_data = QQP(
            data_dir='data',
            split='train',
            create_data=True,
            max_sequence_length=20,
            min_occ=3
        )


Creating new TRAIN qqp data.
Vocablurary of 11042 keys created.


In [30]:
train_data.vocab_size

11042

In [16]:
w2i = dict()
i2w = dict()

special_tokens = ['<pad>', '<unk>', '<sos>', '<eos>']
for st in special_tokens:
    i2w[len(w2i)] = st
    w2i[st] = len(w2i)

In [17]:
i2w

{0: '<pad>', 1: '<unk>', 2: '<sos>', 3: '<eos>'}

In [18]:
w2i

{'<pad>': 0, '<unk>': 1, '<sos>': 2, '<eos>': 3}

In [31]:
import torch
import torch.nn as nn
import torch.nn.utils.rnn as rnn_utils

In [126]:
class VanillaVAE(nn.Module):
    def __init__(self, vocab_size, embedding_size, rnn_type, hidden_size, word_dropout, embedding_dropout, latent_size,
                sos_idx, eos_idx, pad_idx, unk_idx, max_sequence_length, num_layers=1, bidirectional=False):
        super().__init__()
        self.tensor = torch.cuda.FloatTensor if torch.cuda.is_available() else torch.Tensor
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        
        self.max_sequence_length = max_sequence_length
        self.sos_idx = sos_idx
        self.eos_idx = eos_idx
        self.pad_idx = pad_idx
        self.unk_idx = unk_idx
        
        self.latent_size = latent_size
        
        self.rnn_type = rnn_type
        self.bidirectional = bidirectional
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        
        self.embedding = nn.Embedding(vocab_size, embedding_size)
        self.word_dropout_rate = word_dropout
        self.embedding_dropout = nn.Dropout(p=embedding_dropout)
        
        if rnn_type == 'rnn':
            rnn = nn.RNN
        elif rnn_type == 'gru':
            rnn = nn.GRU
        # elif rnn_type == 'lstm':
        #     rnn = nn.LSTM
        else:
            raise ValueError()
            
        self.encoder_rnn = rnn(embedding_size, hidden_size, num_layers=num_layers, bidirectional=self.bidirectional,
                               batch_first=True)
        self.decoder_rnn = rnn(embedding_size, hidden_size, num_layers=num_layers, bidirectional=self.bidirectional,
                               batch_first=True)
        
        self.hidden_factor = (2 if bidirectional else 1) * num_layers
        
        self.hidden2mean = nn.Linear(hidden_size * self.hidden_factor, latent_size)
        self.hidden2logv = nn.Linear(hidden_size * self.hidden_factor, latent_size)
        self.latent2hidden = nn.Linear(latent_size, hidden_size * self.hidden_factor)
        self.outputs2vocab = nn.Linear(hidden_size * (2 if bidirectional else 1), vocab_size)
        
    
    def forward(self, input_sequence, length):
        batch_size = input_sequence.shape[0]
        sorted_lengths, sorted_idx = torch.sort(length, descending=True) # 按长度降序排列
        input_sequence = input_sequence(sorted_idx)
        
        # Encoder
        input_embedding = self.embedding(input_sequence)
        
        packed_input = rnn_utils.pack_padded_sequence(input_embedding, sorted_lengths.data.tolist(), batch_first=True)
        
        _, hidden = self.encoder_rnn(packed_input)
        
        if self.bidirectional or self.num_layers > 1:
            # flatten hidden state
            hidden = hidden.view(batch_size, self.hidden_size*self.hidden_factor)
        else:
            hidden = hidden.squeeze()
            
        # Reparameterization
        mean = self.hidden2mean(hidden)
        logv = self.hidden2logv(hidden)
        std = torch.exp(0.5 * logv)
        
        z = torch.randn([batch_size, self.latent_size]).to(self.device)
        z = z * std + mean
        
        # Decoder 
        hidden = self.latent2hidden(z)

        if self.bidirectional or self.num_layers > 1:
            # unflatten hidden state
            hidden = hidden.view(self.hidden_factor, batch_size, self.hidden_size)
        else:
            hidden = hidden.unsqueeze(0)
        
        ## decoder input
        if self.word_dropout_rate > 0:
            # randomly replace decoder input with <unk>
            prob = torch.rand(input_sequence.shape).to(self.device)
            prob[(input_sequence.data - self.sos_idx) * (input_sequence.data - self.pad_idx) == 0] = 1
            decoder_input_sequence = input_sequence.clone()
            decoder_input_sequence[prob < self.word_dropout_rate] = self.unk_idx
            input_embedding = self.embedding(decoder_input_sequence)
        input_embedding = self.embedding_dropout(input_embedding)
        packed_input = rnn_utils.pack_padded_sequence(input_embedding, sorted_lengths.data.tolist(), batch_first=True)
        
        ## decoder forward pass
        outputs, _ = self.decoder_rnn(packed_input, hidden)

        ## process outputs
        padded_outputs = rnn_utils.pad_packed_sequence(outputs, batch_first=True)[0]
        padded_outputs = padded_outputs.contiguous()
        _, reversed_idx = torch.sort(sorted_idx)
        padded_outputs = padded_outputs[reversed_idx]
        b, s, _ = padded_outputs.size()
        
        ## project outputs to vocab
        logp = nn.functional.log_softmax(self.outputs2vocab(padded_outputs.view(-1, padded_outputs.shape[2])), dim=-1)
        logp = logp.view(b, s, self.embedding.num_embeddings)
        
        return logp, mean, logv, z
    
    def inference(self, n=4, z=None):

        if z is None:
            batch_size = n
            z = torch.randn([batch_size, self.latent_size]).to(self.device)
        else:
            batch_size = z.shape[0]

        hidden = self.latent2hidden(z)

        if self.bidirectional or self.num_layers > 1:
            # unflatten hidden state
            hidden = hidden.view(self.hidden_factor, batch_size, self.hidden_size)

        hidden = hidden.unsqueeze(0)

        # required for dynamic stopping of sentence generation
        sequence_idx = torch.arange(0, batch_size, out=self.tensor()).long()  # all idx of batch
        # all idx of batch which are still generating
        sequence_running = torch.arange(0, batch_size, out=self.tensor()).long()
        sequence_mask = torch.ones(batch_size, out=self.tensor()).bool()
        # idx of still generating sequences with respect to current loop
        running_seqs = torch.arange(0, batch_size, out=self.tensor()).long()

        generations = self.tensor(batch_size, self.max_sequence_length).fill_(self.pad_idx).long()

        t = 0
        while t < self.max_sequence_length and len(running_seqs) > 0:

            if t == 0:
                input_sequence = torch.Tensor(batch_size).fill_(self.sos_idx).long().to(self.device)

            input_sequence = input_sequence.unsqueeze(1)

            input_embedding = self.embedding(input_sequence)

            output, hidden = self.decoder_rnn(input_embedding, hidden)

            logits = self.outputs2vocab(output)

            input_sequence = self._sample(logits)

            # save next input
            generations = self._save_sample(generations, input_sequence, sequence_running, t)

            # update gloabl running sequence
            sequence_mask[sequence_running] = (input_sequence != self.eos_idx)
            sequence_running = sequence_idx.masked_select(sequence_mask)

            # update local running sequences
            running_mask = (input_sequence != self.eos_idx).data
            running_seqs = running_seqs.masked_select(running_mask)

            # prune input and hidden state according to local update
            if len(running_seqs) > 0:
                input_sequence = input_sequence[running_seqs]
                hidden = hidden[:, running_seqs]

                running_seqs = torch.arange(0, len(running_seqs), out=self.tensor()).long()

            t += 1

        return generations, z
    
    def _sample(self, dist, mode='greedy'):

        if mode == 'greedy':
            _, sample = torch.topk(dist, 1, dim=-1)
        sample = sample.reshape(-1)

        return sample

    def _save_sample(self, save_to, sample, running_seqs, t):
        # select only still running
        running_latest = save_to[running_seqs]
        # update token at position t
        running_latest[:,t] = sample.data
        # save back
        save_to[running_seqs] = running_latest

        return save_to

In [127]:
import time
ts = time.strftime('%Y-%b-%d-%H:%M:%S', time.gmtime())

In [128]:
ts

'2022-Feb-18-07:19:06'

In [108]:
rnn = nn.RNN(300, 128, 1, batch_first=True)
input = torch.randn(32, 15, 300)
h0 = torch.randn(1, 32, 128)
output, hn = rnn(input, h0)

In [109]:
output.shape

torch.Size([32, 15, 128])

In [98]:
a = torch.tensor([[1, 4, 56, 32, 43, 2]])
p = torch.rand(a.shape)
p.shape

torch.Size([1, 6])

In [99]:
p

tensor([[0.6438, 0.4026, 0.4299, 0.7945, 0.3941, 0.5233]])

In [100]:
p[(a - 1) * (a - 2) == 0] = 1

In [102]:
p

tensor([[1.0000, 0.4026, 0.4299, 0.7945, 0.3941, 1.0000]])

In [89]:
a[True, False, True] = 1

In [80]:
a

tensor([ 0.0200, -0.0976, -0.3032])

In [65]:
inp = torch.randn(32, 15)

In [69]:
inp.size()

torch.Size([32, 15])

In [33]:
a = torch.randn(2, 3)

In [38]:
a.shape[0]

2

In [115]:
b = torch.LongTensor([[8, 10, 9, 5, 15], [2, 4, 0, 0, 0], [2, 5, 6, 0, 0]])

In [119]:
sorted_lengths, sorted_idx = torch.sort(torch.Tensor([5, 2, 3]), descending=True)

In [130]:
input_sequence = b[sorted_idx]

In [131]:
b

tensor([[ 8, 10,  9,  5, 15],
        [ 2,  4,  0,  0,  0],
        [ 2,  5,  6,  0,  0]])

In [120]:
sorted_lengths

tensor([5., 3., 2.])

In [121]:
sorted_idx

tensor([0, 2, 1])

In [123]:
_, id = torch.sort(sorted_idx)

In [124]:
b[id]

tensor([[ 8, 10,  9,  5, 15],
        [ 2,  5,  6,  0,  0],
        [ 2,  4,  0,  0,  0]])

In [48]:
b.shape

torch.Size([5])

In [50]:
sorted_b = b[sorted_idx]

In [51]:
sorted_b

tensor([15, 10,  9,  8,  5])

In [55]:
sorted_lengths.data.tolist()

[15, 10, 9, 8, 5]

In [62]:
c = rnn_utils.pack_padded_sequence(torch.randn(32, 64), list(range(32, 0, -1)), batch_first=True)

In [2]:
import os
import json
import torch
import argparse

from model import VanillaVAE
from utils import idx2word, interpolate

In [3]:
with open('data/qqp.vocab.json', 'r') as file:
    vocab = json.load(file)

w2i, i2w = vocab['w2i'], vocab['i2w']

os.environ["CUDA_VISIBLE_DEVICES"] = "1"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [62]:
model = VanillaVAE(
        vocab_size=len(w2i),
        sos_idx=w2i['<sos>'],
        eos_idx=w2i['<eos>'],
        pad_idx=w2i['<pad>'],
        unk_idx=w2i['<unk>'],
        max_sequence_length=20,
        embedding_size=300,
        rnn_type='gru',
        hidden_size=256,
        word_dropout=0.,
        embedding_dropout=0.5,
        latent_size=64,
        num_layers=1,
        ).to(device)

In [63]:
# model.load_state_dict(torch.load('checkpoints/E9.pytorch'))
model.load_state_dict(torch.load('checkpoints/word_dropout_0.5/E9.pytorch'))

<All keys matched successfully>

In [64]:
model.eval()

VanillaVAE(
  (embedding): Embedding(11042, 300)
  (embedding_dropout): Dropout(p=0.5, inplace=False)
  (encoder_rnn): GRU(300, 256, batch_first=True)
  (decoder_rnn): GRU(300, 256, batch_first=True)
  (hidden2mean): Linear(in_features=256, out_features=64, bias=True)
  (hidden2logv): Linear(in_features=256, out_features=64, bias=True)
  (latent2hidden): Linear(in_features=64, out_features=256, bias=True)
  (outputs2vocab): Linear(in_features=256, out_features=11042, bias=True)
)

In [31]:
# 加载训练集
with open('data/qqp.train.txt') as f:
    train_data = [line.strip() for line in f]

In [67]:
samples, z = model.inference(n=100)
print('----------SAMPLES----------')
sampled_sentences = idx2word(samples, i2w=i2w, pad_idx=w2i['<pad>'])
sampled_sentences = [' '.join(s.split()[:-1]).strip() for s in sampled_sentences]
print(*sampled_sentences, sep='\n')

----------SAMPLES----------
how can i stop my dog from humping my furniture ?
how long does it take to be <unk> ?
what should i do to beat boredom ?
how will the implementation of gst bill and 1000 notes help to curb black money
can you see who viewed my profile pic on quora ?
how does donald trump realistically win the presidency ?
what is rice puller ?
what should i do to improve my english ?
what are some ways to make money online ?
is it weird to make more mistakes in the morning of the universe ?
what is a woman ?
can you still see who views your vote ?
what is the difference between <unk> and <unk> ? what are they legal in the
which is the best day of your life ?
how can i hack a lost phone ?
what are some arguments for hillary clinton s plans for hillary clinton ?
what is the difference between a barrister and a <unk> ?
how can i get into stanford with a poor gpa ?
what is the use of the <unk> of the united states ?
why did the <unk> <unk> <unk> <unk> <unk> ? how did they get in

In [57]:
n = 0
no_s = []
for s in sampled_sentences:
    if s in train_data:
        n += 1
#         print(s)
    else:
        no_s.append(s)

In [58]:
n

35

In [66]:
for i in range(10):
    samples, z = model.inference(n=1000)
    sampled_sentences = idx2word(samples, i2w=i2w, pad_idx=w2i['<pad>'])
    sampled_sentences = [' '.join(s.split()[:-1]).strip() for s in sampled_sentences]
    
    n = 0
    for s in sampled_sentences:
        if s in train_data:
            n += 1
    print(i, n)

0 84
1 94
2 78
3 102
4 95
5 88
6 98
7 95
8 86
9 109


In [ ]:
# word dropout = 0.
0 418
1 413
2 424
3 402
4 382
5 416
6 430
7 432
8 413
9 415

# word dropout = 0.5
0 84
1 94
2 78
3 102
4 95
5 88
6 98
7 95
8 86
9 109